In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch

# ✅ 최신 모델 선택
summarization_model_name = "facebook/bart-large-cnn"
qa_model_name = "deepset/roberta-base-squad2"

# ✅ 모델 및 토크나이저 로드
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# ✅ GPU 설정 (없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summarization_model = summarization_model.to(device)
qa_model = qa_model.to(device)

# ✅ 파이프라인 설정 (추론 방식 개선)
summarizer = pipeline("summarization", model=summarization_model, tokenizer=summarization_tokenizer, device=0 if torch.cuda.is_available() else -1)
question_answerer = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0 if torch.cuda.is_available() else -1)

# ✅ 긴 문서 입력
long_text = """
Artificial Intelligence (AI) is a field of computer science that aims to create machines capable of intelligent behavior.
AI technology has rapidly evolved over the past decade, making significant contributions in areas such as healthcare, finance, and autonomous systems.
In healthcare, AI assists doctors in diagnosing diseases and recommending personalized treatments.
In finance, AI-driven algorithms optimize stock trading and risk management.
AI-powered self-driving cars are revolutionizing transportation by improving safety and efficiency.
The primary goal of AI research is to develop a deeper understanding of human intelligence and to create machines that can perform complex tasks autonomously.
"""

# ✅ 텍스트 요약 실행 (개선된 요약)
summary = summarizer(long_text, max_length=100, min_length=50, do_sample=True, temperature=0.7)
summary_text = summary[0]['summary_text'].strip()
print("요약된 텍스트:", summary_text, "\n")

# ✅ 질문 입력 및 답변 생성 (질문 개선)
question = "What are the main applications of AI?"
answer = question_answerer(question=question, context=summary_text, top_k=3)
for i, ans in enumerate(answer):
    print(f"{i}번째 AI의 답변:", ans['answer'])


Device set to use cuda:0
Device set to use cuda:0


요약된 텍스트: Artificial Intelligence (AI) is a field of computer science that aims to create machines capable of intelligent behavior. AI technology has rapidly evolved over the past decade, making significant contributions in areas such as healthcare, finance, and autonomous systems. The primary goal of AI research is to develop a deeper understanding of human intelligence and create machines that can perform complex tasks autonomously. 

0번째 AI의 답변: healthcare, finance, and autonomous systems
1번째 AI의 답변: healthcare, finance, and autonomous systems.
2번째 AI의 답변: healthcare, finance


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import torch

# 번역 모델 설정 (한국어 ↔ 영어)
ko_to_en_model_name = "facebook/mbart-large-50-many-to-many-mmt"
en_to_ko_model_name = "facebook/mbart-large-50-many-to-many-mmt"

ko_to_en_tokenizer = AutoTokenizer.from_pretrained(ko_to_en_model_name)
ko_to_en_model = AutoModelForSeq2SeqLM.from_pretrained(ko_to_en_model_name)

en_to_ko_tokenizer = AutoTokenizer.from_pretrained(en_to_ko_model_name)
en_to_ko_model = AutoModelForSeq2SeqLM.from_pretrained(en_to_ko_model_name)

# GPU 설정 (없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ko_to_en_model = ko_to_en_model.to(device)
en_to_ko_model = en_to_ko_model.to(device)

# 번역 파이프라인 설정 (src_lang, tgt_lang 명시적 설정)
translator_ko_to_en = pipeline(
    "translation",
    model=ko_to_en_model,
    tokenizer=ko_to_en_tokenizer,
    src_lang="ko_KR",  # 한국어 → 영어 변환 시 원본 언어
    tgt_lang="en_XX",  # 변환할 대상 언어 (영어)
    device=0 if torch.cuda.is_available() else -1
)

translator_en_to_ko = pipeline(
    "translation",
    model=en_to_ko_model,
    tokenizer=en_to_ko_tokenizer,
    src_lang="en_XX",  # 영어 → 한국어 변환 시 원본 언어
    tgt_lang="ko_KR",  # 변환할 대상 언어 (한국어)
    device=0 if torch.cuda.is_available() else -1
)

# 테스트 문장
long_text_ko = """
인공지능(AI)은 인간의 지능을 모방하여 학습, 추론, 인식 등의 기능을 수행하는 컴퓨터 시스템을 의미합니다.
최근 AI 기술은 머신러닝과 딥러닝의 발전으로 다양한 분야에서 혁신을 이끌고 있습니다.
특히 자연어 처리(NLP) 분야에서는 대규모 언어 모델이 등장하여 인간과 유사한 수준의 언어 이해 및 생성 능력을 보여주고 있습니다.
이러한 기술은 챗봇, 번역, 감정 분석 등 여러 응용 분야에서 활용되고 있습니다.
"""

# Step 1: 한국어 문서를 영어로 번역
translated_text_en = translator_ko_to_en(long_text_ko)[0]['translation_text']
print("번역된 영어 문서:", translated_text_en)

# Step 2: 영어 문서를 다시 한국어로 번역
translated_text_ko = translator_en_to_ko(translated_text_en)[0]['translation_text']
print("번역된 한국어 문서:", translated_text_ko)


Device set to use cuda:0
Device set to use cuda:0


🌍 번역된 영어 문서: Artificial intelligence is a computer system that mimics human intelligence and performs functions such as learning, inference, recognition. Recently, AI has led to innovations in many fields, particularly in the field of natural language processing, where large-scale language models have emerged that demonstrate human-like levels of language comprehension and generation, and these technologies are being used in many applications, including chatbots, translation, emotion analysis.
📢 번역된 한국어 문서: 인공 지능은 인간 지능을 흉내내고 학습, 추론, 인식 같은 기능을 수행하는 컴퓨터 시스템입니다. 최근 AI는 많은 분야에서 혁신을 일으켰습니다. 특히 자연 언어 처리 분야에서 말이죠. 인간과 유사한 수준의 언어 이해와 생성을 보여주는 대규모의 언어 모델이 나타나고, 이 기술은 채트봇, 번역, 감정 분석 등 많은 응용분야에서 사용되고 있습니다.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch

# 최신 모델 선택
summarization_model_name = "facebook/bart-large-cnn"
qa_model_name = "deepset/roberta-base-squad2"

# 모델 및 토크나이저 로드
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# 번역 모델 설정 (한국어 ↔ 영어)
ko_to_en_model_name = "facebook/mbart-large-50-many-to-many-mmt"
en_to_ko_model_name = "facebook/mbart-large-50-many-to-many-mmt"

ko_to_en_tokenizer = AutoTokenizer.from_pretrained(ko_to_en_model_name)
ko_to_en_model = AutoModelForSeq2SeqLM.from_pretrained(ko_to_en_model_name)

en_to_ko_tokenizer = AutoTokenizer.from_pretrained(en_to_ko_model_name)
en_to_ko_model = AutoModelForSeq2SeqLM.from_pretrained(en_to_ko_model_name)

# GPU 설정 (없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summarization_model = summarization_model.to(device)
qa_model = qa_model.to(device)
ko_to_en_model = ko_to_en_model.to(device)
en_to_ko_model = en_to_ko_model.to(device)

# 파이프라인 설정 (src_lang, tgt_lang 명시적 설정)
summarizer = pipeline("summarization", model=summarization_model, tokenizer=summarization_tokenizer, device=0 if torch.cuda.is_available() else -1)
question_answerer = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0 if torch.cuda.is_available() else -1)

translator_ko_to_en = pipeline(
    "translation",
    model=ko_to_en_model,
    tokenizer=ko_to_en_tokenizer,
    src_lang="ko_KR",
    tgt_lang="en_XX",
    device=0 if torch.cuda.is_available() else -1
)

translator_en_to_ko = pipeline(
    "translation",
    model=en_to_ko_model,
    tokenizer=en_to_ko_tokenizer,
    src_lang="en_XX",
    tgt_lang="ko_KR",
    device=0 if torch.cuda.is_available() else -1
)

# 긴 문서 입력 (한국어)
long_text_ko = """
전작에 나온 영웅들의 기술들은 전체적으로 전작의 형태를 유지하고 있지만 기술 개수가 7개에서 11개로 증가해 새로운 기술이 많이 생겼으며, 토큰 시스템이 적용되고 회복 계열 스킬들이 대부분 시전 체력 조건이나 2~3회 정도의 횟수 제한 등 페널티가 생겼으며 장기적인 버프 효과도 많이 없어진 편이라 전작의 감각으로 단편적인 활용을 해서는 고전하기 십상이다. 특히 토큰 시스템에 익숙해지지 못한다면 더욱 어려울 것이다.
또한 전작에 존재하던 시스템인 무기와 방어구 시스템 자체가 사라져 공격 기술에는 자체적으로 공격력이 붙어 있다. 또한 회피율이나 명중률 역시 토큰에 영향을 받기 때문에 토큰이 없다는 전제 하에 기본적인 회피율은 0%, 명중률은 100%이다.
1편에서 등장했던 영웅들이 게임 내에 모두 등장하지는 않았으며, 주인공 일행을 배신하여 도적들과 합류한 유물수집상 등 스토리상으로 출연이 배제된 캐릭터들도 존재한다. 허나 개발자 또한 다른 인터뷰에서 유저들의 전작 영웅들이 출연했으면 하는 요청이 많은 것을 알고 있어 기존 캐릭터를 등장하게 할 생각이 있음을 밝혔기에 새로운 영웅들이 추가될 여지를 남겨두었다.
전작에서는 영웅 고용 시 나레이터가 해당 영웅의 모습과 특징, 출신 등을 이야기했던 것과 다르게 본작에서는 해당 영웅의 과거사를 비추는 말을 한다. 전작과 달리 각 영웅의 과거에 비중을 많이 두었는데, 맵에서 무작위로 등장하는 회고의 성소에서 과거의 메아리를 통해 영웅의 과거사와 스킬을 해금할 수 있다. 다만 모든 영웅이 전작과는 달리 플레이어와 스승과의 연결점이 알려져 있지 않아, 어째서 희망의 횃불을 지키는 입장이 되었는지는 알 수 없다.
현상금 사냥꾼을 제외한 영웅들은 영웅의 길을 4개씩 가지고 있는데 하나는 방랑자라는 공통된 영웅의 길이고 나머지 3개는 영웅별로 정해진 특성화된 영웅의 길이다. 현상금 사냥꾼도 일단은 영웅의 길 자체는 존재하는데 전문가라는 영웅의 길로 고정되어 있다. 영웅의 길은 희망의 제단의 살아있는 도시에서 촛불로 해금해야 한다. 원래는 원정 시작 시 영웅을 결정하는 '교차로'에서 영웅과 사용할 영웅의 길을 정하면 챕터가 끝나도록 계속 고정된 영웅의 길을 사용해야 했지만 1.02.54580 버전부터 여관에서 영웅의 길을 변경할 수 있게 되었다. 계곡 지역의 첫 여관에서는 유물 4, 그 뒤로는 유물 32가 필요하며, 영웅의 길 변경 시 그 영웅에게 투자했던 숙련 포인트는 모두 반납받는다.
전용 장신구는 영웅당 3개이며 전작과 다르게 모두 같은 등급이다. 대부분의 전용 장신구는 영웅의 과거와 관련이 있다. 또한 임시 고용 영웅인 현상금 사냥꾼을 제외한 나머지에게는 전용 여관 아이템이 존재하며 해금이 되었을 시 여관에 입장할 때 확률적으로 얻을 수 있다.
"""

# Step 1: 한국어 문서를 영어로 번역
translated_text_en = translator_ko_to_en(long_text_ko)[0]['translation_text']
print("번역된 영어 문서:", translated_text_en, "\n")

# Step 2: 영어 문서 요약
summary_en = summarizer(translated_text_en, max_length=100, min_length=50, do_sample=True, temperature=0.7)[0]['summary_text']
print("영어 요약된 텍스트:", summary_en, "\n")

# Step 3: 영어 요약을 한국어로 변환
summary_ko = translator_en_to_ko(summary_en)[0]['translation_text']
print("한국어 요약된 텍스트:", summary_ko, "\n")

# Step 4: 한국어 질문을 영어로 번역
question_ko = "전작과 다른 시스템의 특징은 무엇인가요?"
translated_question_en = translator_ko_to_en(question_ko)[0]['translation_text']
print("번역된 영어 질문:", translated_question_en, "\n")

# Step 5: 영어 질문을 영어 요약된 문서에서 질문-응답 수행
answer_en = question_answerer(question=translated_question_en, context=summary_en, top_k=3)

# Step 6: 영어 답변을 다시 한국어로 번역
translated_answers_ko = [translator_en_to_ko(ans['answer'])[0]['translation_text'] for ans in answer_en]

# 첫 번째 답변 출력
print("AI의 첫 번째 답변:", translated_answers_ko[0])

# 모든 답변 출력
for i, ans in enumerate(translated_answers_ko):
    print(f"{i}번째 AI의 답변:", ans)


Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Your input_length: 700 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


번역된 영어 문서: But the developers also revealed in other interviews that they had a lot of requests from their users for their previous heroes to appear, so they left a lot of room for new heroes to be added. In the previous game, the character hire narrator talked about the character's appearance, characteristics, origin, etc., but in the original game, the character hire narrator talks about the character's past. Unlike in the previous game, each hero had a lot of weight in the past, and you could pay for the hero's past and the character's strength through past echoes in the rehearsal chambers that would appear at random on the map, except that all of the heroes had no known connections with the player and the instructor, and you couldn't figure out why they were in charge of keeping the torch of hope. All of the heroes except the 

영어 요약된 텍스트: In the previous game, the character hire narrator talked about the character's appearance, characteristics, origin, etc., but in the original ga

In [6]:

def split_text(text, max_length=512):
    """문장을 일정 길이로 분할"""
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks


def translate_large_text(translator, text, max_length=512):
    """긴 문서를 분할하여 번역 후 합치기"""
    chunks = split_text(text, max_length=max_length)
    translated_chunks = [translator(chunk)[0]['translation_text'] for chunk in chunks]
    return " ".join(translated_chunks).replace("\n", " ").strip()

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch

# 최신 모델 선택
summarization_model_name = "facebook/bart-large-cnn"
qa_model_name = "deepset/roberta-base-squad2"

# 모델 및 토크나이저 로드
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# 번역 모델 설정 (한국어 ↔ 영어)
ko_to_en_model_name = "facebook/mbart-large-50-many-to-many-mmt"
en_to_ko_model_name = "facebook/mbart-large-50-many-to-many-mmt"

ko_to_en_tokenizer = AutoTokenizer.from_pretrained(ko_to_en_model_name)
ko_to_en_model = AutoModelForSeq2SeqLM.from_pretrained(ko_to_en_model_name)

en_to_ko_tokenizer = AutoTokenizer.from_pretrained(en_to_ko_model_name)
en_to_ko_model = AutoModelForSeq2SeqLM.from_pretrained(en_to_ko_model_name)

# GPU 설정 (없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summarization_model = summarization_model.to(device)
qa_model = qa_model.to(device)
ko_to_en_model = ko_to_en_model.to(device)
en_to_ko_model = en_to_ko_model.to(device)

# 파이프라인 설정 (src_lang, tgt_lang 명시적 설정)
summarizer = pipeline("summarization", model=summarization_model, tokenizer=summarization_tokenizer, device=0 if torch.cuda.is_available() else -1)
question_answerer = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0 if torch.cuda.is_available() else -1)

translator_ko_to_en = pipeline(
    "translation",
    model=ko_to_en_model,
    tokenizer=ko_to_en_tokenizer,
    src_lang="ko_KR",
    tgt_lang="en_XX",
    device=0 if torch.cuda.is_available() else -1
)

translator_en_to_ko = pipeline(
    "translation",
    model=en_to_ko_model,
    tokenizer=en_to_ko_tokenizer,
    src_lang="en_XX",
    tgt_lang="ko_KR",
    device=0 if torch.cuda.is_available() else -1
)

# 긴 문서 입력 (한국어)
long_text_ko = """
전작에 나온 영웅들의 기술들은 전체적으로 전작의 형태를 유지하고 있지만 기술 개수가 7개에서 11개로 증가해 새로운 기술이 많이 생겼으며, 토큰 시스템이 적용되고 회복 계열 스킬들이 대부분 시전 체력 조건이나 2~3회 정도의 횟수 제한 등 페널티가 생겼으며 장기적인 버프 효과도 많이 없어진 편이라 전작의 감각으로 단편적인 활용을 해서는 고전하기 십상이다. 특히 토큰 시스템에 익숙해지지 못한다면 더욱 어려울 것이다.
또한 전작에 존재하던 시스템인 무기와 방어구 시스템 자체가 사라져 공격 기술에는 자체적으로 공격력이 붙어 있다. 또한 회피율이나 명중률 역시 토큰에 영향을 받기 때문에 토큰이 없다는 전제 하에 기본적인 회피율은 0%, 명중률은 100%이다.
1편에서 등장했던 영웅들이 게임 내에 모두 등장하지는 않았으며, 주인공 일행을 배신하여 도적들과 합류한 유물수집상 등 스토리상으로 출연이 배제된 캐릭터들도 존재한다. 허나 개발자 또한 다른 인터뷰에서 유저들의 전작 영웅들이 출연했으면 하는 요청이 많은 것을 알고 있어 기존 캐릭터를 등장하게 할 생각이 있음을 밝혔기에 새로운 영웅들이 추가될 여지를 남겨두었다.
전작에서는 영웅 고용 시 나레이터가 해당 영웅의 모습과 특징, 출신 등을 이야기했던 것과 다르게 본작에서는 해당 영웅의 과거사를 비추는 말을 한다. 전작과 달리 각 영웅의 과거에 비중을 많이 두었는데, 맵에서 무작위로 등장하는 회고의 성소에서 과거의 메아리를 통해 영웅의 과거사와 스킬을 해금할 수 있다. 다만 모든 영웅이 전작과는 달리 플레이어와 스승과의 연결점이 알려져 있지 않아, 어째서 희망의 횃불을 지키는 입장이 되었는지는 알 수 없다.
현상금 사냥꾼을 제외한 영웅들은 영웅의 길을 4개씩 가지고 있는데 하나는 방랑자라는 공통된 영웅의 길이고 나머지 3개는 영웅별로 정해진 특성화된 영웅의 길이다. 현상금 사냥꾼도 일단은 영웅의 길 자체는 존재하는데 전문가라는 영웅의 길로 고정되어 있다. 영웅의 길은 희망의 제단의 살아있는 도시에서 촛불로 해금해야 한다. 원래는 원정 시작 시 영웅을 결정하는 '교차로'에서 영웅과 사용할 영웅의 길을 정하면 챕터가 끝나도록 계속 고정된 영웅의 길을 사용해야 했지만 1.02.54580 버전부터 여관에서 영웅의 길을 변경할 수 있게 되었다. 계곡 지역의 첫 여관에서는 유물 4, 그 뒤로는 유물 32가 필요하며, 영웅의 길 변경 시 그 영웅에게 투자했던 숙련 포인트는 모두 반납받는다.
전용 장신구는 영웅당 3개이며 전작과 다르게 모두 같은 등급이다. 대부분의 전용 장신구는 영웅의 과거와 관련이 있다. 또한 임시 고용 영웅인 현상금 사냥꾼을 제외한 나머지에게는 전용 여관 아이템이 존재하며 해금이 되었을 시 여관에 입장할 때 확률적으로 얻을 수 있다.
"""

# Step 1: 한국어 문서를 영어로 번역 (긴 문장 지원)
translated_text_en = translate_large_text(translator_ko_to_en, long_text_ko)
print("기존 한글 문서:", long_text_ko)
print("번역된 영어 문서:", translated_text_en, "\n")

# Step 2: 영어 문서 요약 (요약 길이 최적화)
summary_en = summarizer(translated_text_en, max_length=150, min_length=70, do_sample=True, temperature=0.7)[0]['summary_text']
print("영어 요약된 텍스트:", summary_en, "\n")

# Step 3: 영어 요약을 한국어로 변환 (긴 문장 지원)
summary_ko = translate_large_text(translator_en_to_ko, summary_en)
print("한국어 요약된 텍스트:", summary_ko, "\n")

# Step 4: 한국어 질문을 영어로 번역
question_ko = "전작과 다른 시스템의 특징은 무엇인가요?"
translated_question_en = translator_ko_to_en(question_ko)[0]['translation_text']
print("기존 한글 질문:", question_ko)
print("번역된 영어 질문:", translated_question_en, "\n")

# Step 5: 영어 질문을 영어 요약된 문서에서 질문-응답 수행 (정확도 향상)
answer_en = question_answerer(question=translated_question_en, context=summary_en, top_k=5)

# Step 6: 영어 답변을 다시 한국어로 번역 (긴 문장 지원)
translated_answers_ko = [translate_large_text(translator_en_to_ko, ans['answer']) for ans in answer_en]

# 첫 번째 답변 출력
print("AI의 첫 번째 답변:", translated_answers_ko[0])

# 모든 답변 출력
for i, ans in enumerate(translated_answers_ko):
    print(f"{i+1}번째 AI의 답변:", ans)


Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Your input_length: 248 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 253 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 205 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


기존 한글 문서: 
전작에 나온 영웅들의 기술들은 전체적으로 전작의 형태를 유지하고 있지만 기술 개수가 7개에서 11개로 증가해 새로운 기술이 많이 생겼으며, 토큰 시스템이 적용되고 회복 계열 스킬들이 대부분 시전 체력 조건이나 2~3회 정도의 횟수 제한 등 페널티가 생겼으며 장기적인 버프 효과도 많이 없어진 편이라 전작의 감각으로 단편적인 활용을 해서는 고전하기 십상이다. 특히 토큰 시스템에 익숙해지지 못한다면 더욱 어려울 것이다.
또한 전작에 존재하던 시스템인 무기와 방어구 시스템 자체가 사라져 공격 기술에는 자체적으로 공격력이 붙어 있다. 또한 회피율이나 명중률 역시 토큰에 영향을 받기 때문에 토큰이 없다는 전제 하에 기본적인 회피율은 0%, 명중률은 100%이다.
1편에서 등장했던 영웅들이 게임 내에 모두 등장하지는 않았으며, 주인공 일행을 배신하여 도적들과 합류한 유물수집상 등 스토리상으로 출연이 배제된 캐릭터들도 존재한다. 허나 개발자 또한 다른 인터뷰에서 유저들의 전작 영웅들이 출연했으면 하는 요청이 많은 것을 알고 있어 기존 캐릭터를 등장하게 할 생각이 있음을 밝혔기에 새로운 영웅들이 추가될 여지를 남겨두었다.
전작에서는 영웅 고용 시 나레이터가 해당 영웅의 모습과 특징, 출신 등을 이야기했던 것과 다르게 본작에서는 해당 영웅의 과거사를 비추는 말을 한다. 전작과 달리 각 영웅의 과거에 비중을 많이 두었는데, 맵에서 무작위로 등장하는 회고의 성소에서 과거의 메아리를 통해 영웅의 과거사와 스킬을 해금할 수 있다. 다만 모든 영웅이 전작과는 달리 플레이어와 스승과의 연결점이 알려져 있지 않아, 어째서 희망의 횃불을 지키는 입장이 되었는지는 알 수 없다.
현상금 사냥꾼을 제외한 영웅들은 영웅의 길을 4개씩 가지고 있는데 하나는 방랑자라는 공통된 영웅의 길이고 나머지 3개는 영웅별로 정해진 특성화된 영웅의 길이다. 현상금 사냥꾼도 일단은 영웅의 길 자체는 존재하는데 전문가라는 영웅의 길로 고정되어 있다. 영웅의 길은 희망의 제단의 살아있는 도시에서 촛불